In [1]:
import tensorflow as tf
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

2023-06-06 15:35:32.734522: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 15:35:32.770720: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 15:35:32.771293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 15:35:33.750576: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_path = '../dataset.json'

# Open the file and load its contents
with open(file_path, 'r') as json_file:
    # Read the file content
    file_content = json_file.read()

    # Split the content by newline to get individual JSON objects
    json_objects = file_content.split('\n')

data = []
# Process each JSON object
for json_object in json_objects:
    if json_object.strip() == '':
        continue  # Skip empty lines
    
    # Parse the JSON object
    data.append(json.loads(json_object))

print(len(data))

X = []
Y = []
Y_flat = []

for i in data:
    X.append(np.array(i['vector']))    
    Y.append(np.array(i['matrix']))
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
Y_train_flat =[i.flatten() for i in Y_train]
Y_test_flat =[i.flatten() for i in Y_test]

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [3]:
def get_metrics(y, p_y):
    metrics = {}
    try:
        m = tf.keras.metrics.Accuracy(name="accuracy", dtype=None)
        m.update_state(y, p_y)
        metrics['accuracy'] = m.result().numpy()
    except:
        pass
    try:
        metrics['mse'] = mse(y,p_y)
    except:
        pass
    return metrics
    

# Handmade NN

In [4]:
# Define the neural network architecture
def build_crazy_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(output_dim)
    ])
    return model
def build_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(196)  # Output layer with shape (196,)
    ])
    return model

def build_model_recursive(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Reshape((1, 256)),  # Reshape input to match LSTM requirements
        # tf.keras.layers.LSTM(512, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='relu')),
        tf.keras.layers.LSTM(512),
        tf.keras.layers.Dense(output_dim)
    ])
    return model

# def build_model_convolutional(input_dim):
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
#         tf.keras.layers.Dense(7*7*128, activation='relu'),
#         tf.keras.layers.Reshape((7, 7, 128)),
#         tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2DTranspose(2, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2D(1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Reshape((14, 14, 1))  # Update the output shape to (14, 14, 1)
#     ])
#     return model

In [5]:
# Define the input and output dimensions
input_dim = 128  # Dimensionality of the input vector
output_dim = 196  # Dimensionality of the output vector

In [6]:
# Build the model
model = build_model_recursive(input_dim, output_dim)
model_ann = build_model(input_dim, output_dim)
# model_conv = build_model_convolutional(input_dim)

# Compile the model
model.compile(optimizer='adam', loss='mse')
model_ann.compile(optimizer='adam', loss='mse')
# model_conv.compile(optimizer='adam', loss='mse')

2023-06-06 15:20:47.751904: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-06 15:20:47.770243: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-06 15:20:48.156233: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gra

In [7]:
model_ann.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=10)

Epoch 1/10
32/32 [==============================] - 2s 27ms/step - loss: 1522145.8750
Epoch 2/10
32/32 [==============================] - 1s 26ms/step - loss: 87.3598
Epoch 3/10
32/32 [==============================] - 1s 27ms/step - loss: 87.3365
Epoch 4/10
32/32 [==============================] - 1s 26ms/step - loss: 87.3364
Epoch 5/10
32/32 [==============================] - 1s 27ms/step - loss: 87.3363
Epoch 6/10
32/32 [==============================] - 1s 27ms/step - loss: 87.3363
Epoch 7/10
32/32 [==============================] - 1s 27ms/step - loss: 87.3362
Epoch 8/10
32/32 [==============================] - 1s 27ms/step - loss: 87.3361
Epoch 9/10
32/32 [==============================] - 1s 26ms/step - loss: 87.3360
Epoch 10/10
32/32 [==============================] - 1s 26ms/step - loss: 87.3358


In [8]:
model.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=10)

Epoch 1/10


2023-06-06 15:20:59.331128: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 15:20:59.333532: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 15:20:59.335171: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

32/32 [==============================] - 3s 14ms/step - loss: 87.3061
Epoch 2/10
32/32 [==============================] - 0s 14ms/step - loss: 87.2045
Epoch 3/10
32/32 [==============================] - 0s 14ms/step - loss: 87.0937
Epoch 4/10
32/32 [==============================] - 0s 14ms/step - loss: 87.0138
Epoch 5/10
32/32 [==============================] - 0s 14ms/step - loss: 86.9243
Epoch 6/10
32/32 [==============================] - 0s 14ms/step - loss: 86.8376
Epoch 7/10
32/32 [==============================] - 0s 14ms/step - loss: 86.7833
Epoch 8/10
32/32 [==============================] - 0s 14ms/step - loss: 86.7522
Epoch 9/10
32/32 [==============================] - 0s 14ms/step - loss: 86.7945
Epoch 10/10
32/32 [==============================] - 0s 14ms/step - loss: 86.8160


In [9]:

Y_predicted = model.predict(np.array(X_test))
Y_ann_predicted = model_ann.predict(np.array(X_test))
# Y_conv_predicted = model_conv.predict(np.array(X_test))
print(get_metrics(Y_predicted, Y_test_flat))
print(get_metrics(Y_ann_predicted, Y_test_flat))

11/11 [==============================] - 0s 3ms/step


2023-06-06 15:21:06.305819: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 15:21:06.307804: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 15:21:06.309284: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

11/11 [==============================] - 0s 4ms/step
{'accuracy': 0.0, 'mse': 3724.222143788525}
{'accuracy': 0.0, 'mse': 3726.9294313947094}


In [10]:
Y_predicted = [np.reshape(i, (14,14)) for i in Y_predicted]
# Y_predicted

In [11]:
Y_ann_predicted = [np.reshape(i, (14,14)) for i in Y_ann_predicted]
# Y_ann_predicted

NameError: name 'Y_conv_predicted' is not defined

# AutoKeras

In [13]:
import autokeras as ak
import numpy as np

In [14]:
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
train_set = tf.data.Dataset.from_tensor_slices((X_train, Y_train_flat))
test_set = tf.data.Dataset.from_tensor_slices((X_test, Y_test_flat))


In [ ]:

input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=5000
)
reg.fit(train_set,epochs=10, validation_split=0.15)
predicted_y = reg.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)|
print(reg.evaluate(test_set))
print(get_metrics(Y_test, predicted_y))

Trial 50 Complete [00h 00m 10s]
val_loss: 0.619292140007019

Best val_loss So Far: 0.6093710064888
Total elapsed time: 00h 08m 08s

Search: Running Trial #51

Value             |Best Value So Far |Hyperparameter
False             |False             |structured_data_block_1/normalize
True              |True              |structured_data_block_1/dense_block_1/use_batchnorm
3                 |3                 |structured_data_block_1/dense_block_1/num_layers
16                |16                |structured_data_block_1/dense_block_1/units_0
0.5               |0.5               |structured_data_block_1/dense_block_1/dropout
128               |256               |structured_data_block_1/dense_block_1/units_1
0.25              |0.25              |regression_head_1/dropout
adam              |adam              |optimizer
0.001             |0.001             |learning_rate
64                |64                |structured_data_block_1/dense_block_1/units_2



In [ ]:
regressor = ak.StructuredDataRegressor(max_trials=100, overwrite=True)
regressor.fit(train_set, epochs=10, validation_split=0.15)
predicted_y = regressor.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
print(regressor.evaluate(test_set))

In [ ]:
model = reg.export_model()
model.summary()

In [ ]:
model_regressor = regressor.export_model()
model_regressor.summary()

In [ ]:
X_test = X_test.reshape(-1, 128)
Y_ak_predicted =reg.predict(X_test)

# Autogoal